In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab_Notebooks')


Mounted at /content/drive


In [ ]:
#!git clone https://github.com/wineslab/colosseum-oran-commag-dataset.git

In [ ]:
pwd

'/content/drive/My Drive/Colab_Notebooks'

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 20 23:08:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.02       Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:26:00.0  On |                  N/A |
|  0%   32C    P5    25W / 270W |   1254MiB /  8192MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!pip install tf_agents

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 KB 53.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697658 sha256=9c962f4503fd9ac8248222219a48302ff55407e4b451e12b91b3b9645f40f3f8
  Stored in directory: /root/.cache/pip/wheels/96/b9/bb/994c1324b65e39dd1cd7b8ba92e5fb766dd77980929414a866
Successfully built gym
  Attempting uninstall: pygame
    Found existing installation: pygame 2.3.0
    Uninstalling pygame-2.3.0:
      Successfully uninstalled pygame-2.3.0
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25

In [ ]:
#TPU 사용할 때
!pip uninstall numpy
!pip install numpy==1.23.1

Found existing installation: numpy 1.24.2
Uninstalling numpy-1.24.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.9
    /usr/local/lib/python3.9/dist-packages/numpy-1.24.2.dist-info/*
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libopenblas64_p-r0-15028c96.3.21.so
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.9/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.24.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.6 MB/s eta 0:00:00


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tf_agents.trajectories import time_step as ts

import absl
import time
import os
import glob
import sys


import pandas as pd
import numpy as np

import logging

sys.stdout = open('log.txt', 'w')

# Generating dataset from csv file. Returns a Pandas DataFrame
def entire_dataset_from_single_file(filename,
                                    col_names, # all_metrics_list
                                    selected_col_names, # metric_list_to_extract = ["slice_id", "dl_buffer [bytes]", "tx_brate downlink [Mbps]", "sum_requested_prbs", "sum_granted_prbs"]
                                    remove_zero_req_prb_entries=True,
                                    scale_dl_buffer=True,
                                    replace_zero_with_one=False,
                                    add_prb_ratio=True):
    dataset = pd.read_csv(filename, names=col_names, usecols=selected_col_names, header=0)
    print("-------------------------------------------- entire_dataset_from_single_file 시작 --------------------------------------------")
    print("filename : ", filename)
    print("이전 dataset : ", dataset)
    if remove_zero_req_prb_entries:
        dataset = dataset.loc[dataset['sum_requested_prbs'] > 0].reset_index(drop=True)
    print("remove_zero_req_prb_entries 이후 dataset : ", dataset)
    
    if scale_dl_buffer and any(["dl_buffer [bytes]" in m for m in # in: 데이터 안에 찾고자 하는 것이 있는지 확인하는 연산자
                                selected_col_names]):
        # scale the dl_buffer
        dataset['dl_buffer [bytes]'] = dataset['dl_buffer [bytes]'] / 100000
    
    print("scale_dl_buffer 이후 dataset : ", dataset)

    if add_prb_ratio:
        dict_add = pd.DataFrame.from_dict({"ratio_granted_req": np.clip(np.nan_to_num(  # from_dict :  dict객체로부터 DataFrame 객체로 변환하는 메서드
            dataset["sum_granted_prbs"] / dataset["sum_requested_prbs"]), a_min=0, a_max=1) # numpy.clip(array, min, max)
        })
        print("dict_add : ", dict_add)
        print("-------------------------------------------- entire_dataset_from_single_file 끝 --------------------------------------------")

        if replace_zero_with_one: # 실행 안됨
            print("replace_zero_with_one 실행 : ")
            dict_add['ratio_granted_req'].loc[dataset['sum_requested_prbs'] <= 0] = 1.0
        return dataset.join(dict_add)
    else:
        return dataset

# return all csv files inside a single DataFrame
def entire_dataset_from_folder(main_folder,
                               wildcard,
                               col_names, # all_metrics_list
                               selected_col_names, # metric_list_to_extract = ["slice_id", "dl_buffer [bytes]", "tx_brate downlink [Mbps]", "sum_requested_prbs", "sum_granted_prbs"]
                               scale_dl_buffer=True,
                               remove_zero_req_prb_entries=True,
                               replace_zero_with_one=False,
                               add_prb_ratio=True):
  
    dataset = []
    for filename in glob.glob(main_folder + wildcard):
        db_tmp = entire_dataset_from_single_file(filename, col_names=col_names,
                                                 selected_col_names=selected_col_names,
                                                 scale_dl_buffer=scale_dl_buffer,
                                                 remove_zero_req_prb_entries=remove_zero_req_prb_entries,
                                                 replace_zero_with_one=replace_zero_with_one,
                                                 add_prb_ratio=add_prb_ratio)
                                                 
        dataset.append(db_tmp)
    
    return pd.concat(dataset, axis=0, ignore_index=True)


# take n entries from the DataFrame at random
def extract_n_entries_from_dataset(dataset=None,
                                   slice_id=None,
                                   n_entries=10,
                                   metrics_export=None):
    if slice_id is not None:
        d_temp = dataset.loc[dataset['slice_id'] == int(slice_id)]
    else:
        d_temp = dataset

    d_temp = d_temp.sample(n=n_entries).reset_index(drop=True)
    if metrics_export is not None:
        d_temp = d_temp[metrics_export]

    return d_temp

# This function is used here to emulate a DU reporting real-time data. Replace this function with your DU
# FOR TESTING PURPOSES ONLY
def get_data_from_DUs(dataset=None,
                      n_entries=1000,
                      n_col=4,
                      slice_id=None,
                      metrics_export=None):

    if dataset is None:  # generate random data in case you do not have a dataset
        values = np.random.random(size=(n_entries, n_col))
        slice_id = np.random.randint(low=0, high=3, size=(n_entries, 1))
        data = np.concatenate((slice_id, values), axis=1)
    else:
        data = extract_n_entries_from_dataset(dataset=dataset,
                                              slice_id=slice_id,
                                              n_entries=n_entries,
                                              metrics_export=metrics_export)

    return data


# Return lists for metrics, rewards, prbs assigned to each slice.
# Ideally, the list is such that len(list) = num_slices
def split_data(slice_profiles=None, # slice_profiles
               data_to_spit=None, # data_to_spit=data
               metric_list=None, # metric_dict=metric_dict
               metric_dict=None, # metric_list=metric_list_for_agents
               n_entries_per_slice=None): # n_entries_per_slice=n_entries_for_autoencoder
    print("----------------------------------------split_data 시작----------------------------------------")
    print("slice_profiles : ", slice_profiles)
    print("data_to_spit : ", data_to_spit)
    print("metric_list : ", metric_list)
    print("metric_dict : ", metric_dict)
    print("n_entries_per_slice : ", n_entries_per_slice)
    
    
    metrics = []
    prbs = []
    rewards = []
    
    # ordering here follows slice_profiles
    for i in slice_profiles:

        slice_data = data_to_spit[data_to_spit[:, metric_dict['slice_id']] == slice_profiles[i]['slice_id'], :]
        print("slice_data")
        if slice_data.size > 0:
            # repmat on rows to reach needed dimension in case you do not have enough reporting data
            while slice_data.shape[0] < n_entries_per_slice:
                slice_data = np.vstack((slice_data, np.zeros((1, slice_data.shape[1]))))
                print("slice_data in while :", slice_data)
            print("slice_data out of while :", slice_data)

            slice_prb = slice_data[:, metric_dict['slice_prb']]
            slice_metrics = slice_data[:, [metric_dict[x] for x in metric_list]]
            slice_reward = slice_data[:, metric_dict[slice_profiles[i]['reward_metric']]]
            print("slice_prb : ", slice_prb)
            print("slice_metrics : ", slice_metrics)
            print("slice_reward : ", slice_reward)
            

            if n_entries_per_slice is not None:
                slice_prb = slice_prb[0:n_entries_per_slice]
                slice_metrics = slice_metrics[0:n_entries_per_slice, :]
                slice_reward = slice_reward[0:n_entries_per_slice]
                print("if문 slice_prb : ", slice_prb)
                print("if문 slice_metrics : ", slice_metrics)
                print("if문 slice_reward : ", slice_reward)
        else:
            slice_metrics = []
            slice_prb = []
            slice_reward = []

        metrics.append(slice_metrics)
        prbs.append(slice_prb)
        rewards.append(slice_reward)
        print("metrics : ", metrics)
        print("prbs : ", prbs)
        print("rewards : ", rewards)
        
    print("----------------------------------------split_data 끝----------------------------------------")
    return metrics, prbs, rewards

# Used to generate the input to the DRL agent. It returns a TimeStep that contains (step_type, reward, discount, observations)
def generate_timestep_for_policy(obs_tmp=None):
    print("-------------------------------generate_timestep_for_policy 시작-------------------------------")
    step_type = tf.convert_to_tensor(
        [0], dtype=tf.int32, name='step_type')
    print("step_type : ", step_type)
    reward = tf.convert_to_tensor(
        [0], dtype=tf.float32, name='reward')
    print("reward : ", reward)
    discount = tf.convert_to_tensor(
        [1], dtype=tf.float32, name='discount')
    print("discount : ", discount)
    observations = tf.convert_to_tensor(
        [obs_tmp], dtype=tf.float32, name='observations')
    print("observations : ", observations)
    print("-------------------------------generate_timestep_for_policy 끝-------------------------------")
    return ts.TimeStep(step_type, reward, discount, observations)

if __name__ == '__main__':
    print("----------------------------main함수 시작----------------------------")
    # Column names in the srsLTE CSV dataset
    all_metrics_list = ["Timestamp",
                        "num_ues",
                        "IMSI",
                        "RNTI",
                        "empty_1",
                        "slicing_enabled",
                        "slice_id",
                        "slice_prb",
                        "power_multiplier",
                        "scheduling_policy",
                        "empty_2",
                        "dl_mcs",
                        "dl_n_samples",
                        "dl_buffer [bytes]",
                        "tx_brate downlink [Mbps]",
                        "tx_pkts downlink",
                        "tx_errors downlink (%)",
                        "dl_cqi",
                        "empty_3",
                        "ul_mcs",
                        "ul_n_samples",
                        "ul_buffer [bytes]",
                        "rx_brate uplink [Mbps]",
                        "rx_pkts uplink",
                        "rx_errors uplink (%)",
                        "ul_rssi",
                        "ul_sinr",
                        "phr",
                        "empty_4",
                        "sum_requested_prbs",
                        "sum_granted_prbs",
                        "empty_5",
                        "dl_pmi",
                        "dl_ri",
                        "ul_n",
                        "ul_turbo_iters"]

    # Column names we need to extract from the dataset
    metric_list_to_extract = ["slice_id",
                              "dl_buffer [bytes]",
                              "tx_brate downlink [Mbps]",
                              "sum_requested_prbs",
                              "sum_granted_prbs"]

    # configure logger and console output
    logging.basicConfig(level=logging.DEBUG, filename='./agent.log', filemode='a+',
                        format='%(asctime)-15s %(levelname)-8s %(message)s')
    formatter = logging.Formatter('%(asctime)-15s %(levelname)-8s %(message)s')
    console = logging.StreamHandler() #FileHandler('./agent.log')
    console.setLevel(logging.INFO)
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)

    use_gpu_in_env = True
    mtc_policy_filename = './ml_models/mtc_policy'
    urllc_policy_filename = './ml_models/urllc_policy'
    embb_policy_filename = './ml_models/embb_policy'
    autoencoder_filename = './ml_models/encoder.h5'

    # Location of the dataset we want to use (valid in offline testing ONLY)
    main_folder = './slice_traffic/rome_static_close/tr10'
    wildcard_match = '/*/*/slices_bs*/*_metrics.csv'
    # get dataset for testing purposes only.
    # This is used as this code does not run with hardware components.
    # Not needed if getting data from real DUs
    dataset = entire_dataset_from_folder(main_folder=main_folder,
                                         wildcard=wildcard_match,
                                         col_names=all_metrics_list,
                                         selected_col_names=metric_list_to_extract)
    print("dataset : ", dataset)
    # Input size to the autoencoder for dimentionality reduction
    n_entries_for_autoencoder = 10

    # set logging level + enable TF2 behavior
    absl.logging.set_verbosity(absl.logging.INFO)
    # select which GPU to use
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    if use_gpu_in_env is False:
        gpu_devices = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)
        print("Num GPUs Available outside environments: ", len(gpu_devices))

    # load policy, these are the folder where saved_model.pb is stored
    drl_agents = [tf.saved_model.load(embb_policy_filename),
                  tf.saved_model.load(mtc_policy_filename),
                  tf.saved_model.load(urllc_policy_filename)]
    print("drl_agents : ", drl_agents)
    absl.logging.info('Agents loaded')

    autoencoder = tf.keras.models.load_model(autoencoder_filename, compile=False)

    absl.logging.info('Autoencoder loaded')

    slice_profiles = {'embb': {'slice_id': 0, 'reward_metric': "tx_brate downlink [Mbps]"},
                      'mtc': {'slice_id': 1, 'reward_metric': "tx_brate downlink [Mbps]"},
                      'urllc': {'slice_id': 2, 'reward_metric': "ratio_granted_req"}}

    metric_dict = {"dl_buffer [bytes]": 1,
                   "tx_brate downlink [Mbps]": 2,
                   "ratio_granted_req": 3,
                   "slice_id": 0,
                   "slice_prb": 4}

    metric_list_for_agents = ["dl_buffer [bytes]",
                   "tx_brate downlink [Mbps]",
                   "ratio_granted_req"]

    default_policy = 0
    previous_policy = dict()
    for _, val in slice_profiles.items():
        previous_policy[val['slice_id']] = default_policy
    print("previous_policy : ", previous_policy)

    previous_metrics = ''
    print("----------------------------main함수끝----------------------------")

    while True:
        print("------------------------------------------------while 시작------------------------------------------------")
        policies = list()

        # This is where data comes from the DUs.
        # As an example, we extract data from the static dataset.
        # Users may want to interface it with their own DUs
        data = get_data_from_DUs(dataset=dataset,
                                 n_entries=1000,
                                 metrics_export=metric_list_to_extract).to_numpy()
        print("data : ", data)

        data_tmp, prbs, rewards = split_data(slice_profiles=slice_profiles,
                                             data_to_spit=data,
                                             metric_dict=metric_dict,
                                             metric_list=metric_list_for_agents,
                                             n_entries_per_slice=n_entries_for_autoencoder)
        print("data_tmp : ", data_tmp, "prbs : ", prbs, "rewards : ", rewards)
        print("slice_profiles : ", slice_profiles)
        for i in range(len(slice_profiles)):
            print("len(slice_profiles) : ", len(slice_profiles)) #-> 3
            if len(data_tmp[i]) > 0:
                print("len(data_tmp[i]) :", len(data_tmp[i])) #-> 10
                for row in data_tmp[i]:
                    row[0] /= 100000

                logging.info('Testing iteration ' + str(i))
                print("Testing iteration " + str(i))
                logging.info('Data received from DU (dl_buffer [bytes], tx_brate downlink [Mbps], ratio_granted_req): ')
                print("Data received from DU (dl_buffer [bytes], tx_brate downlink [Mbps], ratio_granted_req): ")
                print("np.expand_dims(data_tmp[i], axis=0) : ", np.expand_dims(data_tmp[i], axis=0))
                logging.info(np.expand_dims(data_tmp[i], axis=0))

                obs_tmp = autoencoder.predict(np.expand_dims(data_tmp[i], axis=0)).astype('float32')
                print("obs_tmp 1: ", obs_tmp)
                print("prbs[", i, "][0] : ", prbs[i][0])
                obs_tmp = np.append(obs_tmp, prbs[i][0]).astype('float32')
                print("obs_tmp 2: ", obs_tmp)

                print("rewards[", i, "] : ", rewards[i])
                reward_mean = np.mean(rewards[i]).astype('float32')
                print("reward_mean : ", reward_mean)
                time_step = generate_timestep_for_policy(obs_tmp)
                print("time_step ", time_step)
                action = drl_agents[i].action(time_step)
                print("action ", action)

                # append policies to send and store policy
                print("action[0][0][0].numpy() : ", action[0][0][0].numpy())
                policies.append(action[0][0][0].numpy())
                previous_policy[i] = action[0][0][0].numpy()

                logging.info('Slice ' + str(i) + ': Action is ' + str(action[0][0][0].numpy()) + ' Reward is: ' + str(reward_mean))
                print("Slice "+ str(i) + ": Action is " + str(action[0][0][0].numpy()) + " Reward is: " + str(reward_mean))
            else:
                # append previous policy
                policies.append(previous_policy[i])
                logging.info("Using previous action " + str(previous_policy[i]) + " for slice profile " + str(i))
                print("Using previous action " + str(previous_policy[i]) + " for slice profile " + str(i))

        # build message to send policies to the DU
        msg = ','.join([str(x) for x in policies])
        logging.info("Sending this message to the DU: " + msg)
        print("Sending this message to the DU: " + msg)
        print("------------------------------------------------while 끝------------------------------------------------")

        time.sleep(10)

        # Users may want to plug in their own functions to send the DRL policies
        # to the DU based on the specific DU implementation in use
        # send_action_to_DU(DU_address, msg)


ValueError: No objects to concatenate

In [ ]:
#from absl import logging
import logging
import absl
absl.logging.info('Interesting Stuff')
absl.logging.info('Interesting Stuff with Arguments: %d', 42)

absl.logging.set_verbosity(logging.INFO)
absl.logging.log(logging.DEBUG, 'This will *not* be printed')
absl.logging.set_verbosity(logging.DEBUG)
absl.logging.log(logging.DEBUG, 'This will be printed')

absl.logging.warning('Worrying Stuff')
absl.logging.error('Alarming Stuff')
absl.logging.fatal('AAAAHHHHH!!!!')

INFO:absl:Interesting Stuff
2023-03-29 02:45:34,294 INFO     Interesting Stuff
2023-03-29 02:45:34,294 INFO     Interesting Stuff
2023-03-29 02:45:34,294 INFO     Interesting Stuff
2023-03-29 02:45:34,294 INFO     Interesting Stuff
2023-03-29 02:45:34,294 INFO     Interesting Stuff
INFO:absl:Interesting Stuff with Arguments: 42
2023-03-29 02:45:34,302 INFO     Interesting Stuff with Arguments: 42
2023-03-29 02:45:34,302 INFO     Interesting Stuff with Arguments: 42
2023-03-29 02:45:34,302 INFO     Interesting Stuff with Arguments: 42
2023-03-29 02:45:34,302 INFO     Interesting Stuff with Arguments: 42
2023-03-29 02:45:34,302 INFO     Interesting Stuff with Arguments: 42
Level 1:absl:This will *not* be printed
Level 1:absl:This will be printed
2023-03-29 02:45:34,309 WARNING  Worrying Stuff
2023-03-29 02:45:34,309 WARNING  Worrying Stuff
2023-03-29 02:45:34,309 WARNING  Worrying Stuff
2023-03-29 02:45:34,309 WARNING  Worrying Stuff
2023-03-29 02:45:34,309 WARNING  Worrying Stuff
ERROR: